In [18]:
import google.generativeai as genai

In [19]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access environment variables
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Create/Connect to a local database
For example, we'll use Soka Mysql DB.

Load the sql IPython extension so you can interact with the database using magic commands (the % instructions) to create a new, empty SQLite database.

In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect,text

In [4]:
# Define your database connection credentials
user = 'root'
password = ''
host = 'localhost'
port = '3306'  # Default is usually 3306
database = 'lepont'

# Create the connection string
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
# Create the database engine
engine = create_engine(connection_string)
inspector = inspect(engine)

In [5]:
def list_tables() -> list[str]:
    print(' - DB CALL: list_tables')
    return inspector.get_table_names()

list_tables()

 - DB CALL: list_tables


['account',
 'branch',
 'customer',
 'customer_account',
 'customer_followups',
 'customer_to_users',
 'eod_branch',
 'loan_account',
 'loan_account_emergency',
 'main_account',
 'migrations',
 'rp_track_total_asset',
 'transaction',
 'user']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [6]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    
    """Look up the table schema.
    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')
    return [ (col['name'],str(col['type'])[:11]) for col in inspector.get_columns(table_name)]

describe_table("loan_account")

 - DB CALL: describe_table


[('id', 'INTEGER'),
 ('customer_id', 'INTEGER'),
 ('account_id', 'INTEGER'),
 ('principal_amount', 'DECIMAL(15,'),
 ('loan_date', 'DATE'),
 ('due_date', 'DATE'),
 ('completed_date', 'DATE'),
 ('status', 'ENUM'),
 ('flag', 'ENUM'),
 ('created_at', 'DATETIME'),
 ('updated_at', 'DATETIME')]

In [7]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    with engine.connect() as connection:
        # Execute the raw SQL query
        result = connection.execute(text(sql))
        # Fetch all rows and convert them to a list of lists
        return [(row) for row in result]

# Example usage
results = execute_query("SELECT * FROM customer")
results[0]

 - DB CALL: execute_query


(291, 'Mariam Ahmadi Rashid', 'F', '0746288306', '', 'Boma Road', 'Mkaa Na Mabarafu', 'Boma road', None, datetime.date(1975, 1, 9), 'NIDA', '19750109671020000114', 1, None, datetime.datetime(2022, 1, 15, 13, 41, 43), datetime.datetime(2022, 5, 29, 10, 50, 59), 'uploads/Mariam_Ahmadi_Rashid_0746288306_20220115104143.jpg', 19, 1)

In [8]:
results[0]

(291, 'Mariam Ahmadi Rashid', 'F', '0746288306', '', 'Boma Road', 'Mkaa Na Mabarafu', 'Boma road', None, datetime.date(1975, 1, 9), 'NIDA', '19750109671020000114', 1, None, datetime.datetime(2022, 1, 15, 13, 41, 43), datetime.datetime(2022, 5, 29, 10, 50, 59), 'uploads/Mariam_Ahmadi_Rashid_0746288306_20220115104143.jpg', 19, 1)

## Implement function calls
Now you can put it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a function_call instead. When this happens, the client must respond with a function_response, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports automatic function calling, where the supplied functions will be automatically invoked. This is a powerful feature and should only be exposed when it is safe to do so, such as when the functions have no side-effects.


In [9]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

# instruction = """You are a helpful chatbot that can interact with an SQL database for a African Leagues and Competitions. You will take the users questions and turn them into SQL queries using the tools
# available. Once you have the information you need, you will answer the user's question using
# the data returned. Use list_tables to see what tables are present, describe_table to understand
# the schema, and execute_query to issue an SQL SELECT query. The all events related to match are stored in match_events, while all events related to team are stored in match_team_events. The events related to player are stored in match_player_events"""

instruction = """You are a helpful chatbot that can interact with an SQL database for a microfinance company. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query. The account_id in loan_account connect to account which link details related to branches. Amount taken is similar to principal_amount. Loans are stored in loan_account. 
Cast final answers from sql queries to string"""


model = genai.GenerativeModel(
    "models/gemini-1.5-flash-latest", tools=db_tools, system_instruction=instruction
)

# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)

In [10]:
resp = chat.send_message("Which branch name has the most loans", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: execute_query
The branch with the most loans is Morogoro, with 2793 loans.


In [11]:
resp = chat.send_message("What about Kihonda?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query


ProgrammingError: (pymysql.err.ProgrammingError) (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'\\"Kihonda\\" GROUP BY b.name\' at line 1')
[SQL: SELECT b.name , COUNT(la.account_id) as loan_count FROM loan_account la JOIN account a ON la.account_id = a.id JOIN branch b ON a.branch_id = b.id WHERE b.name = \"Kihonda\" GROUP BY b.name]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [12]:
resp = chat.send_message("What are the available branches", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
The available branches are Morogoro, HQ, Kihonda, and Mbagala.



In [13]:
resp = chat.send_message("Kihonda total customers?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
There are 0 customers in the Kihonda branch.



In [113]:
resp = chat.send_message("What competitions are available??", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
Based on the available tables,  I can tell you about the competitions. To give you the most accurate answer, I need to know the specific information you are looking for.  Do you want a list of all competitions, or are you interested in specific details like the competition name, season, or participating teams?



In [115]:
resp = chat.send_message("List all competition names", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
Here's a list of all the competition names:

Premier League, First Division League, Second Division League, CECAFA KAGAME CHAMPIONSHIP, CHAMPIONS LEAGUE, CAF CONFEDERATION, CHAMPIONS PRIMARY STAGE, Azam Federation cup, Ngao ya jamii, African Cup of Nations Qualification, Mapinduzi Cup, SportPesa Super Cup, Afcon-U17, Africa Cup of Nations, Kenya premier league, KENYA FA CUP, CECAFA



In [123]:
resp = chat.send_message("Which match had most goals", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
Match with ID 918 had the most goals.


In [129]:
resp = chat.send_message("List available tables", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
The available tables are: administrators, assists, board_members, club_details, coach_teams, coaches, comments, competition_participants, competition_rules, competition_sponsors, competition_winners, competitions, competitions_tagged_news, countries, countries2, cup_groups, cup_hosts, cup_matches, cups, divisions, documents, events, leadership_positions, league_groups, league_matches, league_officers, leagues, levels, match_comments, match_events, match_lineups, match_officials, match_photos, match_player_events, match_team_events, matches, migrations, news, news_comments, news_likes, news_media, news_views, notification_items, password_resets, player_teams, players, players_tagged_news, roles, rules, seasons, social_media, sponsors, stadiums, substitutions, suspended_matches, suspensions, team_contacts, team_kits, team_social_media, teams, teams_tagged_news, transfer_windows, transfers, user_favourite_notifications, user_favourites, user_general_notifications, 

In [131]:
resp = chat.send_message("Yanga SC", request_options=retry_policy)
print(resp.text)

To answer your question about Yanga SC, I need to know what you would like to know about them.  Do you want to know their match results, their current players, their standing in a league, or something else?



In [132]:
resp = chat.send_message("Match Results", request_options=retry_policy)
print(resp.text)

To get Yanga SC's match results, I need to know what season or competition you are interested in.  Can you specify a season (e.g., 2022-2023) or a specific competition (e.g., Tanzanian Premier League)?



In [133]:
resp = chat.send_message("Premier League for season 2019", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query


OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'home_team' in 'where clause'")
[SQL: SELECT * FROM matches WHERE home_team = 'Yanga SC' OR away_team = 'Yanga SC' AND season = 2019]
(Background on this error at: https://sqlalche.me/e/20/e3q8)